### Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import os
from os.path import join
from skimage import color, io

if not os.path.exists('imgs'):
    os.makedirs('imgs')

### Speed transfers in relation to start velocity and phase of interacting bees

In [2]:
def plot_binned_speed_changes(matrix, save_to=None, binning_var='start_vel',
                              null=False):
    
    matplotlib.rcParams.update({'font.size': 5, 'ytick.major.size': 1,
                                'ytick.major.width': 0.4, 'font.sans-serif': 'Arial'})
    
    fig, axs = plt.subplots(1, 1, figsize=(1.67, 1.67), dpi=300)
    
    # Set labels and title.
    label = 'Speed change of focal bee [mm/s]'
    bin_label = 'Start velocity' if binning_var == 'start_vel' else 'Time of peak activity\n'
    title = 'Null-model' if null else 'Real data'
    unit = f'[$n$th 6-quantile]'
    
    # Colorbar settings.
    cbar_kws = {'label': label, 'shrink': 0.8}
    if binning_var == 'phase':
        cbar_kws['ticks'] = [0.04, 0.05]
    else:
        cbar_kws['ticks'] = [-0.9,-0.6,-0.3,0,0.3]
        
    # Set constant colormap range for real data and null-model.
    color_range = (-1.02, 0.4) if binning_var == 'start_vel' else (0.0395, 0.055)
    
    # Plot.
    sns.heatmap(matrix, annot=False, cmap='viridis', robust=True, ax=axs,
                cbar_kws=cbar_kws, linewidths=0, alpha=0.95,
                vmin=color_range[0], vmax=color_range[1])
    
    axs.invert_yaxis()
    axs.set(xlabel=f'{bin_label} of focal bee {unit}',
            ylabel=f'{bin_label} of non-focal bee {unit}',
            title=title)
    axs.tick_params(size=0)

    # Save as vector graphic.
    if save_to:
        plt.savefig(save_to + '.svg', bbox_inches='tight')
    plt.show()    